## Imports

In [4]:
import os
import sys
import time
import shutil
import random
from pathlib import Path
from tqdm import tqdm
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from datetime import datetime
import time
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader, WeightedRandomSampler, TensorDataset
from torchvision import models, datasets, transforms
from PIL import Image
from collections import Counter

c:\Users\deepesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## Data Prep

In [ ]:
def split_dataset(source_dir, output_dir, train_ratio=0.8, seed=42, type='balanced'):
    random.seed(seed)

    categories = [cat for cat in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, cat))]
    if type != 'balanced':
        random.shuffle(categories)
    summary = {}

    print(f"\n📁 Splitting dataset from: {source_dir}")
    print(f"💾 Output directory: {output_dir}\n")

    for idx, category in enumerate(categories):
        print(idx, category)
        category_path = os.path.join(source_dir, category)
        images = os.listdir(category_path)
        random.shuffle(images)

        split_idx = int(len(images) * train_ratio)

        train_images = images[:split_idx]
        test_images = images[split_idx:]

        if type == 'unbalanced':
            max_train_images = split_idx - (idx * 35)
            train_images = train_images[:max_train_images]
            print(len(train_images), max_train_images)

        summary[category] = {
            'train': len(train_images),
            'test': len(test_images)
        }

        print(f"\n🔹 Splitting category: {category}")
        for split_name, split_images in [('train', train_images), ('test', test_images)]:
            split_path = Path(output_dir) / split_name / category
            split_path.mkdir(parents=True, exist_ok=True)

            for img in tqdm(split_images, desc=f"   ➤ {split_name.upper()} [{category}]", ncols=80):
                src = Path(category_path) / img
                dst = split_path / img
                try:
                    shutil.copyfile(src, dst)
                except Exception as e:
                    print(f"⚠️ Could not copy {img}: {e}")

    # Final Summary
    print("\n📊 Split Summary:")
    total_train, total_test = 0, 0
    for category, stats in summary.items():
        print(f"   {category}: {stats['train']} train / {stats['test']} test")
        total_train += stats['train']
        total_test += stats['test']

    print("\n✅ All categories processed successfully!")
    print(f"🗂️  Total training images: {total_train}")
    print(f"🧪 Total testing images:  {total_test}")
    print(f"\n📍 Check your data at: {output_dir}/train/ and {output_dir}/test/\n")

In [9]:
def evaluate_classification(y_true, y_pred, class_names=None, verbose=True):
    """
    Evaluate classification performance and optionally print a report.
    """
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision_macro": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall_macro": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
    }

    if verbose:
        print("\n📊 Classification Report:")
        print(classification_report(y_true, y_pred, target_names=class_names))

    return metrics

In [8]:
def plot_confusion_matrix(y_true, y_pred, class_names, normalize=False, figsize=(10, 8), title="Confusion Matrix", save_path=None):
    """
    Plot and optionally save a confusion matrix.
    """
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt='.2f' if normalize else 'd', cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"📁 Saved confusion matrix to {save_path}")
    else:
        plt.show()

In [7]:
CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)

## ML Methods

In [5]:
def extract_sift_features(image_path, max_descriptors=500):
    try:
        img = cv2.imread(image_path, 0)
        sift = cv2.SIFT_create(nfeatures=max_descriptors)
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is None:
            return np.zeros((max_descriptors, 128)).flatten()

        if descriptors.shape[0] > max_descriptors:
            descriptors = descriptors[:max_descriptors]
        else:
            pad = np.zeros((max_descriptors - descriptors.shape[0], 128))
            descriptors = np.vstack((descriptors, pad))

        return descriptors.flatten()
    except:
        return None

In [11]:
def prepare_data(data_dir, max_descriptors=500, use_pca=True, pca_dim=300, type='balanced'):
    cache_name = f"{'pca' if use_pca else 'nopca'}_{os.path.basename(data_dir)}_sift_{max_descriptors}.npz"
    cache_path = os.path.join(CACHE_DIR, type, cache_name)
    os.makedirs(f"{CACHE_DIR}/{type}", exist_ok=True)

    if os.path.exists(cache_path):
        print(f"\nLoading cached features from {cache_path}")
        cache = np.load(cache_path, allow_pickle=True)
        return cache["X"], cache["y"], cache["labels"].tolist(), cache["pca"] if "pca" in cache.files else None

    print(f"\nExtracting features from: {data_dir}")
    X, y = [], []
    label_encoder = LabelEncoder()

    for label in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label)
        if not os.path.isdir(label_path): continue

        files = os.listdir(label_path)
        print(f"{label} - {len(files)} images")

        for file in tqdm(files, desc=f"   Extracting [{label}]", ncols=80):
            image_path = os.path.join(label_path, file)
            feat = extract_sift_features(image_path, max_descriptors)
            if feat is not None:
                X.append(feat)
                y.append(label)

    X = np.array(X)
    y = label_encoder.fit_transform(y)
    labels = label_encoder.classes_.tolist()

    pca_obj = None
    if use_pca:
        print("Applying PCA to reduce dimensionality...")
        pca_obj = PCA(n_components=pca_dim)
        # pca_obj = PCA()
        X = pca_obj.fit_transform(X)

    print(f"Caching features to {cache_path}")
    np.savez_compressed(cache_path, X=X, y=y, labels=labels, pca=pca_obj)

    return X, y, labels, pca_obj

In [11]:
def train_svm(X_train, y_train, type='balanced'):
    print("\nTraining SVM classifier (SIFT + SVM)... This may take a few minutes.")
    start_time = time.time()
    
    clf = SVC(kernel='linear', probability=True, verbose=1)
    clf.fit(X_train, y_train)

    duration = time.time() - start_time
    print(f"Training complete. Time taken: {duration:.2f} seconds.")

    # Save model
    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    model_path = f"models/{type}/svm_classifier.joblib"
    joblib.dump(clf, model_path)
    print(f"Model saved to: {model_path}")

    return clf

In [12]:
def train_sgd(X_train, y_train, type='balanced'):
    print("\nTraining SGDClassifier (Linear SVM approximation)...")
    start_time = time.time()

    clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, verbose=1)
    clf.fit(X_train, y_train)
    
    duration = time.time() - start_time
    print(f"Training complete. Time taken: {duration:.2f} seconds.")

    # Save the model
    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    model_path = f"models/{type}/sgd_classifier.joblib"
    joblib.dump(clf, model_path)
    print(f"Model saved to: {model_path}")

    return clf

In [50]:
def train_rf(X_train, y_train, type='balanced'):
    print("\nTraining Random Forest classifier...")
    start_time = time.time()

    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=1)
    clf.fit(X_train, y_train)

    duration = time.time() - start_time
    print(f"Training complete. Time taken: {duration:.2f} seconds.")
    
    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    model_path = f"models/{type}/sgd_classifier.joblib"
    joblib.dump(clf, model_path)
    print(f"Model saved to: {model_path}")

    return clf

In [51]:
def evaluate(clf, X_test, y_test, labels, method, type='balanced'):
    print("\nEvaluating model...")

    preds = clf.predict(X_test)
    report = classification_report(y_test, preds, target_names=labels, digits=4)
    metrics = evaluate_classification(y_test, preds, labels, verbose=False)

    timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    result_dir = os.path.join("results", type, "ML_results", f"{method}_{timestamp}")
    os.makedirs(result_dir, exist_ok=True)
    os.makedirs(f"{result_dir}/{type}", exist_ok=True)

    report_file = os.path.join(result_dir, type, "report.txt")
    with open(report_file, "w", encoding="utf-8") as f:
        f.write(f"Evaluation Timestamp: {timestamp}\n")
        f.write(f"Model: SIFT + {method}\n")
        f.write("\nSummary Metrics:\n")
        for k, v in metrics.items():
            f.write(f"  {k}: {v:.4f}\n")
        f.write("\nFull Classification Report:\n")
        f.write(report)

    print(f"Report saved to: {report_file}")

    cm_path = os.path.join(result_dir, "confmat.png")
    plot_confusion_matrix(
        y_true=y_test,
        y_pred=preds,
        class_names=labels,
        normalize=True,
        title=f"SIFT + {method} Confusion Matrix",
        save_path=cm_path
    )
    print(f"Confusion matrix saved to: {cm_path}")

    print(report)

## DL Methods

In [38]:
def get_class_weights(dataset):
    counts = Counter(dataset.targets)
    num_samples = sum(counts.values())
    weights = [num_samples / counts[i] for i in range(len(counts))]
    return torch.FloatTensor(weights)

In [40]:
def get_data_loaders(data_dir, batch_size=32, type="balanced"):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    train_ds = datasets.ImageFolder(os.path.join(data_dir, type, 'train'), transform=train_transform)
    test_ds = datasets.ImageFolder(os.path.join(data_dir, type, 'test'), transform=test_transform)


    if type != 'balanced':
        # Count instances of each class
        class_counts = Counter(train_ds.targets)
        class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
        sample_weights = [class_weights[label] for label in train_ds.targets]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

        train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler)
    else:
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    

    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, train_ds.classes

In [14]:
def train_efficientnet(train_loader, num_classes, num_epochs=5, lr=0.001, type='balanced', class_weights=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsing device: {device}")

    model = EfficientNet.from_pretrained('efficientnet-b0')
    in_features = model._fc.in_features
    model._fc = nn.Linear(in_features, num_classes)
    model = model.to(device)

    if class_weights is not None:
        class_weights = class_weights.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        start_time = time.time()
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100)
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        avg_loss = running_loss / len(train_loader)
        epoch_time = time.time() - start_time
        print(f"\n✅ Epoch {epoch+1} completed — Avg Loss: {avg_loss:.4f} ⏱️ {epoch_time:.2f}s")

    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    torch.save(model.state_dict(), f"models/{type}/efficientnet_b0.pth")
    print(f"Model saved to: models/{type}/efficientnet_b0.pth")

    return model

In [15]:
def train_resnet(train_loader, num_classes, num_epochs=5, lr=0.001, type='balanced', class_weights=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_name = torch.cuda.get_device_name(0) if device.type == "cuda" else "CPU"
    print(f"\n🖥️  Training on: {device} ({device_name})")

    if device.type == "cuda":
        print(f"🧠 CUDA Memory Allocated: {torch.cuda.memory_allocated() // (1024 ** 2)} MB")

    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False  # freeze feature extractor

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    if class_weights is not None:
        class_weights = class_weights.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = optim.Adam(model.fc.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        start_time = time.time()

        progress_bar = tqdm(train_loader, desc=f"🔁 Epoch {epoch+1}/{num_epochs}", ncols=100)
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        avg_loss = running_loss / len(train_loader)
        epoch_time = time.time() - start_time
        print(f"\n✅ Epoch {epoch+1} completed — Avg Loss: {avg_loss:.4f} ⏱️ {epoch_time:.2f}s")

    # Save model
    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    torch.save(model.state_dict(), f"models/{type}/resnet18_finetuned.pth")
    print(f"💾 Model saved to: models/{type}/resnet18_finetuned.pth")

    return model

In [16]:
def train_mobilenet(train_loader, num_classes, num_epochs=5, lr=0.001, type='balanced', class_weights=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsing device: {device}")

    model = models.mobilenet_v2(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False  # freeze backbone

    model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    model = model.to(device)

    if class_weights is not None:
        class_weights = class_weights.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = optim.Adam(model.classifier.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        start_time = time.time()
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100)

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        avg_loss = running_loss / len(train_loader)
        epoch_time = time.time() - start_time
        print(f"\n✅ Epoch {epoch+1} completed — Avg Loss: {avg_loss:.4f} ⏱️ {epoch_time:.2f}s")

    os.makedirs("models", exist_ok=True)
    os.makedirs(f"models/{type}", exist_ok=True)
    torch.save(model.state_dict(), f"models/{type}/mobilenet_v2.pth")
    print(f"Model saved to: models/{type}/mobilenet_v2.pth")
    return model


In [17]:
def evaluate_model(model, test_loader, class_names, method, type='balanced'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
    metrics = evaluate_classification(all_labels, all_preds, class_names, verbose=False)

    timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    result_dir = os.path.join("results", type, "DL_results", f"{method}_eval_{timestamp}")
    os.makedirs(result_dir, exist_ok=True)

    report_file = os.path.join(result_dir, "report.txt")
    with open(report_file, "w", encoding="utf-8") as f:
        f.write(f"Evaluation Timestamp: {timestamp}\n")
        f.write(f"Model: {method}\n")
        f.write(f"Device: {device}\n\n")
        f.write("Summary Metrics:\n")
        for k, v in metrics.items():
            f.write(f"  {k}: {v:.4f}\n")
        f.write("\nFull Classification Report:\n")
        f.write(report)

    print(f"Report saved to: {report_file}")

    cm_path = os.path.join(result_dir, "confmat.png")
    plot_confusion_matrix(
        y_true=all_labels,
        y_pred=all_preds,
        class_names=class_names,
        normalize=True,
        title=f"{method} Confusion Matrix",
        save_path=cm_path
    )

    print(f"Confusion matrix saved to: {cm_path}")
    print(f"\nClassification Report ({method}):")
    print(report)

## Model Run

In [ ]:
def run_methods(type='balanced'):
    # split_dataset('archive/Aerial_Landscapes', f"data/{type}", type=type)
    # X_train, y_train, class_names, _ = prepare_data(f"data/{type}/train", use_pca=True, type=type)
    # X_test, y_test, _, _ = prepare_data(f"data/{type}/test", use_pca=True, type=type)

    # clf_sgd = train_sgd(X_train, y_train, type=type)
    # clf_svm = train_svm(X_train, y_train, type=type)
    # clf_rf = train_rf(X_train, y_train, type=type)
    
    # evaluate(clf_sgd, X_test, y_test, class_names, 'SGD', type=type)
    # evaluate(clf_svm, X_test, y_test, class_names, 'SVM', type=type)
    # evaluate(clf_rf, X_test, y_test, class_names, 'RandomForest', type=type)


    train_loader, test_loader, class_names = get_data_loaders('data', batch_size=32)
    if type != 'balanced':
        weights = get_class_weights(train_loader.dataset)

    model_efficientnet = train_efficientnet(train_loader, num_classes=len(class_names), num_epochs=5, type=type, class_weights=weights)
    # model_resnet = train_resnet(train_loader, num_classes=len(class_names), num_epochs=5, type=type, class_weights=weights)
    # model_mobilenet = train_mobilenet(train_loader, num_classes=len(class_names), num_epochs=5, type=type, class_weights=weights)
    
    evaluate_model(model_efficientnet, test_loader, class_names, 'Efficientnet', type=type)
    # evaluate_model(model_resnet, test_loader, class_names, 'Resnet', type=type)
    # evaluate_model(model_mobilenet, test_loader, class_names, 'Mobilenet', type=type)

In [21]:
run_methods()


📁 Splitting dataset from: archive/Aerial_Landscapes
💾 Output directory: data/balanced


🔹 Splitting category: Agriculture


   ➤ TEST [Agriculture]: 100%|██████████████| 160/160 [00:00<00:00, 1697.63it/s]



🔹 Splitting category: Airport


   ➤ TEST [Airport]: 100%|██████████████████| 160/160 [00:00<00:00, 1940.51it/s]



🔹 Splitting category: Beach


...

   Extracting [Residential]: 100%|████████████| 640/640 [11:13<00:00,  1.05s/it]


River - 640 images


   Extracting [River]: 100%|██████████████████| 640/640 [01:53<00:00,  5.65it/s]


Applying PCA to reduce dimensionality...
Caching features to cache\balanced\pca_train_sift_500.npz


FileNotFoundError: [Errno 2] No such file or directory: 'cache\\balanced\\pca_train_sift_500.npz'

In [ ]:
run_methods('unbalanced')


Loading cached features from cache\unbalanced\pca_train_sift_500.npz

Loading cached features from cache\unbalanced\pca_test_sift_500.npz

Using device: cpu
Loaded pretrained weights for efficientnet-b0


Epoch 1/5:  21%|███████▊                              | 62/300 [06:25<37:25,  9.43s/it, Loss=0.4557]